In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import uuid

from datasets import Dataset

In [24]:
class WhatWhereWhenContentParser:
    def __init__(self, urls: list[str]) -> None:
        self.urls = urls

        self.dataset = []

    def parse(self):
        for url in tqdm(self.urls):
            response = requests.get(url=url)
            soup = BeautifulSoup(response.text)
            all_tasks = soup.find_all("div", class_="question")

            # editor = soup.find_all("div", class_="editor")
            # if len(editor) > 0:
            #     editor = editor[0].text
            # else:
            #     editor = soup.find("h1", "info")

            # title = soup.find("h1", "title").text
            # date = soup.find_all("p")[1].text[5:].strip()

            for task in all_tasks:
                has_image = len(task.find_all("img")) > 0
                has_razdatka = len(task.find_all("div", class_="razdatka")) > 0
                has_razdatka2 = "Раздаточный материал" in task.text
                has_explanation = len(task.find_all("strong", class_="Comments")) > 0
                if (
                    not has_image
                    and not has_razdatka
                    and has_explanation
                    and not has_razdatka2
                ):
                    answer = task.find_all("p")[1].text.replace("\nОтвет:", "").strip()

                    question = task.find_all("p")[0].text
                    question = re.sub(r"Вопрос \d+\:", "", question).strip()

                    explanation = (
                        task.find("strong", class_="Comments")
                        .find_parent("p")
                        .text.replace("Комментарий:", "")
                        .strip()
                    )

                    self.dataset.append(
                        {
                            # "title": title,
                            # "date": date,
                            # "editor": editor,
                            "question": question,
                            "answer": answer,
                            "explanation": explanation,
                            "url": url,
                        }
                    )

        save_path = "./verbalist/datasets/what_where_when/what_where_when_questions.csv"
        pd.DataFrame(data=self.dataset).to_csv(save_path, index=False)


links_dataset = pd.read_csv(
    "./verbalist/datasets/what_where_when/what_where_when_links.csv"
)
links_dataset = links_dataset["link"].tolist()

parser = WhatWhereWhenContentParser(urls=links_dataset)
parser.parse()

100%|██████████| 5397/5397 [2:02:07<00:00,  1.36s/it]  


In [36]:
dataset = pd.read_csv(
    "./verbalist/datasets/what_where_when/what_where_when_questions.csv"
)

dataset["uuid"] = dataset["explanation"].apply(lambda x: str(uuid.uuid4()))
dataset = dataset.astype("str")
dataset = dataset.to_dict("records")
dataset = Dataset.from_list(dataset)
# dataset.push_to_hub("dim/what_where_when_ru")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:21<00:00, 21.54s/it]


### parse links


In [61]:
from tqdm import tqdm


class WhatWhereWhenLinksParser:
    def __init__(
        self,
    ) -> None:
        self.urls = []

        self.dataset = []
        self.domain = "https://db.chgk.info"

    def parse(
        self,
    ):
        pages_amount = 270
        for page_num in tqdm(range(pages_amount)):
            url = f"https://db.chgk.info/last?page={page_num}"
            response = requests.get(url=url)
            soup = BeautifulSoup(response.text)

            odd_links = soup.find_all("tr", "odd")
            even_links = soup.find_all("tr", "even")
            links = even_links + odd_links

            for link in links:
                document_link = link.find_all("a")[0]["href"]
                document_link = f"{self.domain}{document_link}"
                self.dataset.append(document_link)


links_parser = WhatWhereWhenLinksParser()
links_parser.parse()

100%|██████████| 270/270 [01:14<00:00,  3.64it/s]


In [63]:
len(links_parser.dataset)

5397

In [64]:
dataset = [{"link": item} for item in links_parser.dataset]

In [67]:
pd.DataFrame(data=dataset).to_csv(
    "./verbalist/datasets/what_where_when/what_where_when_links.csv", index=False
)

### create subset for instruction tuning


In [1]:
from datasets import load_dataset

dataset = load_dataset("dim/what_where_when_ru")
dataset = dataset["train"]

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 222.26it/s]


Dataset parquet downloaded and prepared to /home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--what_where_when_ru-fc08f0f8242e77d0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 179.08it/s]


In [2]:
dataset = dataset.train_test_split(test_size=3000, seed=42)

In [3]:
dataset['test'].to_csv("./verbalist/datasets/what_where_when/what_where_when_selected.csv")

Creating CSV from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 29.37ba/s]


2515768

In [4]:
dataset['test'].push_to_hub('dim/what_where_when_3k')

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  4.60it/s]
Updating downloaded metadata with the new split.


In [2]:
from datasets import load_dataset

dataset = load_dataset("dim/what_where_when_3k")
dataset = dataset['train']

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--what_where_when_3k-15e437af05cc5b3b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 864.27it/s]


In [3]:
dataset[0]

{'question': 'Дуплет.\n\n\xa0\xa0\xa0\xa01. Джон Стюарт Милль считал, что ОНА стала первым примером ущерба,\nкоторый может нанести общество, не способное предоставить полную свободу\nречи, мысли и действия. Назовите ЕЕ двумя словами, начинающимися на одну\nи ту же букву.\n\n\xa0\xa0\xa0\xa02. ОНА произошла в тот день, когда "Коринтианс" выиграл чемпионат\nБразилии, и, таким образом, пожелание сбылось. Назовите ЕЕ двумя\nсловами, начинающимися на одну и ту же букву.',
 'answer': '1. Смерть Сократа.\n\n\xa0\xa0\xa0\xa02. Смерть Сократеса.',
 'explanation': '1. Одной из причин смертного приговора Сократу являлось обвинение в\nбогохульстве.\n\n\xa0\xa0\xa0\xa02. Известный футболист Сократес однажды выразил желание уйти из жизни\nв воскресенье, когда "Коринтианс" выиграет трофей.',
 'url': 'https://db.chgk.info/tour/kaluga16',
 'uuid': 'f1e0b73c-abc0-47f3-9d78-d73b2a38bea9'}

In [5]:
print(dataset[0]['question'])

Дуплет.

    1. Джон Стюарт Милль считал, что ОНА стала первым примером ущерба,
который может нанести общество, не способное предоставить полную свободу
речи, мысли и действия. Назовите ЕЕ двумя словами, начинающимися на одну
и ту же букву.

    2. ОНА произошла в тот день, когда "Коринтианс" выиграл чемпионат
Бразилии, и, таким образом, пожелание сбылось. Назовите ЕЕ двумя
словами, начинающимися на одну и ту же букву.


In [6]:
print(dataset[0]['explanation'])

1. Одной из причин смертного приговора Сократу являлось обвинение в
богохульстве.

    2. Известный футболист Сократес однажды выразил желание уйти из жизни
в воскресенье, когда "Коринтианс" выиграет трофей.


In [7]:
print(dataset[0]['answer'])

1. Смерть Сократа.

    2. Смерть Сократеса.
